In [11]:
# Import required packages
import numpy as np  
import pandas as pd 
import glob
import os
import json
from pandas.io.json import json_normalize
from utils import open_jsons, json_to_df

ImportError: cannot import name randbits

In [ ]:
# Create a list of JSON files
streaming_jsons = ['StreamingHistory0', 'StreamingHistory1', 'StreamingHistory2']
search_jsons = ['SearchQueries']

In [ ]:
# Take a look at the JSONs
open_jsons("StreamingHistory0")

In [47]:
# Define the file path to the directory containing the JSON files
json_files_dir = 'Data/MyData/'

# Define a list of file names for the JSON files
streaming_files = ['StreamingHistory0.json', 'StreamingHistory1.json', 'StreamingHistory2.json']

# Use an F-string to construct the file paths to the JSON files
streaming_files = [f'{json_files_dir}{file_name}' for file_name in streaming_files]

# Convert the JSON files to a Dataframe and concatenate them
df_stream = json_to_df(streaming_files)

# Show dataframe
df_stream.head()


,endTime,artistName,trackName,msPlayed
0,2021-12-01 09:47,Pa Salieu,Energy (feat. Mahalia),120106
1,2021-12-01 10:22,Peter von Poehl,The Story of the Impossible,2450
2,2021-12-01 10:23,21 Savage,Intro,70044
3,2021-12-01 10:24,21 Savage,Runnin,46705
4,2021-12-01 10:25,21 Savage,Intro,70044


In [48]:
# create a 'UniqueID' for each song by combining the fields 'artistName' and 'trackName'
df_stream['UniqueID'] = df_stream['artistName'] + "_" + df_stream['trackName']
df_stream.head(5)

,endTime,artistName,trackName,msPlayed,UniqueID
0,2021-12-01 09:47,Pa Salieu,Energy (feat. Mahalia),120106,Pa Salieu:Energy (feat. Mahalia)
1,2021-12-01 10:22,Peter von Poehl,The Story of the Impossible,2450,Peter von Poehl:The Story of the Impossible
2,2021-12-01 10:23,21 Savage,Intro,70044,21 Savage:Intro
3,2021-12-01 10:24,21 Savage,Runnin,46705,21 Savage:Runnin
4,2021-12-01 10:25,21 Savage,Intro,70044,21 Savage:Intro


In [ ]:
# Opening JSON file
f = open('Data/MyData/Follow.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
data 

In [49]:
# open library JSON
with open(f"{json_files_dir}YourLibrary.json") as json_data:
    data = json.load(json_data)

# create library dataframe 
df_library = pd.DataFrame(data['tracks'])


# add UniqueID column (same as above)
df_library['UniqueID'] = df_library['artist'] + ":" + df_library['track']

# add column with track URI stripped of 'spotify:track:'
new = df_library["uri"].str.split(":", expand = True)
df_library['track_uri'] = new[2]

# take a look at new dataframe
df_library.head()

,artist,album,track,uri,UniqueID,track_uri
0,Calvin Harris,C.U.B.A,C.U.B.A,spotify:track:1ZoHuLxbQho7N5ioKA6Hx1,Calvin Harris:C.U.B.A,1ZoHuLxbQho7N5ioKA6Hx1
1,EARTHGANG,Proud Of U (feat. Young Thug),Proud Of U (feat. Young Thug),spotify:track:6SIR9Zx1I9VaFDPj99z9JW,EARTHGANG:Proud Of U (feat. Young Thug),6SIR9Zx1I9VaFDPj99z9JW
2,Mike WiLL Made-It,Ransom,California Rari,spotify:track:26NxEw6oH3DwwL4o251liP,Mike WiLL Made-It:California Rari,26NxEw6oH3DwwL4o251liP
3,JAY-Z,Watch The Throne,Illest Motherf**ker Alive,spotify:track:4HWQexsnWXsAKBJp52hU4G,JAY-Z:Illest Motherf**ker Alive,4HWQexsnWXsAKBJp52hU4G
4,Loyle Carner,"Not Waving, But Drowning",Angel,spotify:track:4pjcxC0qgXWZWmcBjzn1az,Loyle Carner:Angel,4pjcxC0qgXWZWmcBjzn1az


In [50]:
# create final dict as a copy df_stream
df_spotify = df_stream.copy()

# add column checking if streamed song is in library
# not used in this project but could be helpful for cool visualizations
df_spotify['In Library'] = np.where(df_spotify['UniqueID'].isin(df_library['UniqueID'].tolist()),1,0)

# left join with df_library on UniqueID to bring in album and track_uri
df_spotify = pd.merge(df_spotify, df_library[['album','UniqueID','track_uri']],how='left',on=['UniqueID'])

df_spotify.tail()

,endTime,artistName,trackName,msPlayed,UniqueID,In Library,album,track_uri
28282,2022-12-01 19:43,A$AP Rocky,"Purple Swag REMIX (feat. Bun B, Paul Wall & Ki...",252373,"A$AP Rocky:Purple Swag REMIX (feat. Bun B, Pau...",0,NaN,NaN
28283,2022-12-01 19:45,Migos,Kelly Price (feat. Travis Scott),103514,Migos:Kelly Price (feat. Travis Scott),1,Culture,204U3zjm9kkzX25Pr4Rdld
28284,2022-12-01 19:46,Juicy J,Neighbor (feat. Travis Scott),49899,Juicy J:Neighbor (feat. Travis Scott),0,NaN,NaN
28285,2022-12-01 19:47,Kid Cudi,Baptized In Fire,62217,Kid Cudi:Baptized In Fire,0,NaN,NaN
28286,2022-12-01 19:52,Kid Cudi,The Guide,306440,Kid Cudi:The Guide,0,NaN,NaN


In [56]:
import pandas as pd

def drop_rows_with_keywords(df, keywords):
    # Create a list of rows to drop
    rows_to_drop = []

    # Loop over the rows of the Dataframe
    for i, row in df.iterrows():
        # Loop over the columns of the Dataframe
        for col in df.columns:
            # Loop over the keywords
            for keyword in keywords:
                # Check if the cell contains the keyword
                if keyword in str(df.loc[i, col]):
                    # If the cell contains the keyword, add the row to the list of rows to drop
                    rows_to_drop.append(i)

    # Drop the rows from the Dataframe
    df = df.drop(rows_to_drop)

    return df


# Define a list of keywords
keywords = ['call', 'her daddy']

# Drop the rows containing the keywords
df_clean = drop_rows_with_keywords(df_spotify, keywords)


In [59]:
df_clean.to_excel("spotify_data.xlsx", sheet_name='spotify_data_clean')  

In [70]:
# Look for specific rows
contain_values = df_clean[df_clean['trackName'].str.contains('')]
print (contain_values)

                endTime        artistName       trackName  msPlayed  \
69     2021-12-02 17:38   Astrud Gilberto        Berimbau    145226   
1017   2021-12-17 08:54   Astrud Gilberto        Berimbau     12454   
1867   2022-01-01 10:30   Astrud Gilberto        Berimbau    145226   
1905   2022-01-01 13:00    Daniel Rateuke         Marimbo      1536   
3325   2022-02-02 19:22  Flatbush Zombies      Big Shrimp    237604   
...                 ...               ...             ...       ...   
24767  2022-10-22 08:20        Kanye West  Violent Crimes     39886   
25280  2022-10-25 09:43        Kanye West  Violent Crimes    215320   
26304  2022-11-04 12:23        Kanye West  Violent Crimes    215320   
26996  2022-11-13 13:17        Kanye West  Violent Crimes     36426   
27368  2022-11-19 18:31        Kanye West  Violent Crimes    215320   

                          UniqueID  In Library             album  \
69        Astrud Gilberto:Berimbau           0               NaN   
1017      A

### Additional Data from Spotify for Developers

In [6]:
# generate access token
import requests
import os

# load dotenv file that contains app secrets
from dotenv import load_dotenv
load_dotenv()

# authentication URL
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': os.getenv("CLIENT_ID"),
    'client_secret': os.getenv("CLIENT_SECRET"),
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [7]:
# used for authenticating all API calls
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

In [8]:
# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

In [9]:
# create blank dictionary to store track URI, artist URI, and genres
dict_genre = {}

# convert track_uri column to an iterable list
track_uris = df_library['track_uri'].to_list()

# loop through track URIs and pull artist URI using the API,
# then use artist URI to pull genres associated with that artist
# store all these in a dictionary
for t_uri in track_uris:
    
    dict_genre[t_uri] = {'artist_uri': "", "genres":[]}
    
    r = requests.get(BASE_URL + 'tracks/' + t_uri, headers=headers)
    r = r.json()
    a_uri = r['artists'][0]['uri'].split(':')[2]
    dict_genre[t_uri]['artist_uri'] = a_uri
    
    s = requests.get(BASE_URL + 'artists/' + a_uri, headers=headers)
    s = s.json()
    dict_genre[t_uri]['genres'] = s['genres']

NameError: name 'df_library' is not defined

In [27]:
# convert dictionary into dataframe with track_uri as the first column
df_genre = pd.DataFrame.from_dict(dict_genre, orient='index')
df_genre.insert(0, 'track_uri', df_genre.index)
df_genre.reset_index(inplace=True, drop=True)

df_genre.head()

,track_uri,artist_uri,genres
0,1ZoHuLxbQho7N5ioKA6Hx1,7CajNmpbOovFoOoasH2HaY,"[dance pop, edm, electro house, house, pop, pr..."
1,6SIR9Zx1I9VaFDPj99z9JW,5MbNzCW3qokGyoo9giHA3V,"[atl hip hop, hip hop, psychedelic hip hop, ra..."
2,26NxEw6oH3DwwL4o251liP,0NWbwDZY1VkRqFafuQm6wk,"[atl hip hop, pop rap, rap, southern hip hop, ..."
3,4HWQexsnWXsAKBJp52hU4G,3nFkdlSjzX9mRTtwJOzDYB,"[east coast hip hop, hip hop, rap]"
4,4pjcxC0qgXWZWmcBjzn1az,4oDjh8wNW5vDHyFRrDYC4k,"[indie soul, london rap, uk alternative hip hop]"


In [28]:
df_genre_expanded = df_genre.explode('genres')
df_genre_expanded.head()

,track_uri,artist_uri,genres
0,1ZoHuLxbQho7N5ioKA6Hx1,7CajNmpbOovFoOoasH2HaY,dance pop
0,1ZoHuLxbQho7N5ioKA6Hx1,7CajNmpbOovFoOoasH2HaY,edm
0,1ZoHuLxbQho7N5ioKA6Hx1,7CajNmpbOovFoOoasH2HaY,electro house
0,1ZoHuLxbQho7N5ioKA6Hx1,7CajNmpbOovFoOoasH2HaY,house
0,1ZoHuLxbQho7N5ioKA6Hx1,7CajNmpbOovFoOoasH2HaY,pop


In [29]:
# save df_tableau and df_genre_expanded as csv files that we can load into Tableau
df_spotify.to_csv('MySpotifyDataTable.csv')
df_genre_expanded.to_csv('GenresExpandedTable.csv')

print('done')

done
